## Introduction


This document outlines a procedure for conducting A/B/n Bayesian testing based on the approach described in [VWO's 
white paper](https://cdn2.hubspot.net/hubfs/310840/VWO_SmartStats_technical_whitepaper.pdf).
 The Python implementation is available in [here](https://stash.aexp.com/stash/projects/ONE-STREAM/repos/xp-algorithms/browse/sequential-testing/vwo).  
  
A binary-outcome test (e.g. a test of conversion rates) can be modeled as a binomial sequence with parameter $p$. The beta distribution is a convenient prior distribution for modeling a binomial parameter $p$. Starting from a non-informative prior $Beta\left(1, 1\right)$, the posterior probability distribution of $p$ after $N$ events with $S$ successes is given by $Beta\left(S+1,N-S+1\right)$ accoring to Bayes theorem. Therefore, throughout the test, we only need to track only 2 numbers to compute the posterior distribution - $N$ an $S$.

Once we can derive the probability distribution for a test, we can easily compute many useful statistics. 
For example, we can compute the probability that the variant $A$ is better than the variant $B$ using the joint
probability distrbution.

$$X:\,Random\,variable\,for\:conversion\,rate\,of\,variant\,A\,$$
$$\,Y:\,Random\,variable\,for\,conversion\,rate\,of\,variant\,B$$

$$X\sim Beta\left(a,b\right),Y\sim Beta\left(c,d\right)$$

$$P\left(X>Y\right)= \int_0^1\int_y^1 \frac{x^a\left(1-x\right)^b}{B\left(a,b\right)} \frac{y^c\left(1-y\right)^d}{B\left(c,d\right)}~dx~dy 
\equiv g\left(a, b, c, d\right)
= \frac{\Gamma\left(d+b\right)\Gamma\left(d+c\right)}{\Gamma\left(d+b+c\right)\Gamma\left(d\right)} + \sum_{i=1}^{a-1} \left(\frac{\Gamma\left(i+c\right)\Gamma\left(b+d\right)\Gamma\left(i+b\right)\Gamma\left(c+d\right)}{\Gamma\left(i\right)\Gamma\left(b\right)\Gamma\left(c\right)\Gamma\left(d\right)\Gamma\left(i+b+c+d\right)}\right) / i$$
   
$$B\left(a, b\right)\,is\,the\,beta\,fuction\,with\,parameters\,a\,and\,b$$
$$\Gamma\left(a\right)\,is\,the\,gamma\,fuction\,with\,parameter\,a$$
   
Note:  The derivation of the above closed-form formula can be found [here](https://www.johndcook.com/UTMDABTR-005-05.pdf). 

Here is VWO's dashboard for reporting test results which includes many additonal statistics.

<img src="https://static.wingify.com/gcp/wp-content/themes/vwo/images/new-website/experiments/smartstats2.svg">

Source: https://vwo.com/resources/bayesian-ab-testing/

These statistics can be reproduced by the following code:

In [1]:
import util.ab_core as ab_core
ab_core.calc_stat(sa=250, na=11221, sb=1600, nb=14187)

{'[sa, na, sb, nb]': [250, 11221, 1600, 14187],
 '[alpha_a, beta_a, alpha_b, beta_b]': [251, 10972, 1601, 12588],
 'cvr_a': 0.022364786598948586,
 'cvr_b': 0.11283388540418635,
 'ci_cvr_a': [0.01892988453955432, 0.02611928740825431],
 'ci_cvr_b': [0.10609539604453345, 0.11977736047524634],
 'prob_a_gt_b': 7.432095440609216e-190,
 'prob_b_gt_a': 1.0,
 'lift_b_vs_a': 0.09046909880523776,
 'el_a': 0.0904690988100059,
 'el_b': 8.835834986583339e-194,
 'el_res': 'B'}

## Running a Bayesian A/B test at a fixed sample size

We can use the process similar to the frequentist approach to run a Bayesian A/B test at a fixed sample size. First, determine the desired $\alpha, \beta$, minimum detectable effect and the baseline conversion rate. Then use this  information to compute the required sample size. Here is an example:

In [2]:
import util.helper as helper
helper.sample_size(alpha=0.05, beta=0.05, p0=0.02, mde=0.05, variants=2)

Sample size for alpha=0.050, beta=0.050, p0=0.020, mde=0.050(lift=0.001):
Each variant=521,855, Total=1,043,709


VWO define the loss function and expected loss as below. Expected loss represents the expected losses associated with 
choosing "A" or "B" as the winner. 

**Loss function:**
  
$L_x=max\left(Y-X, 0\right),L_y=max\left(X-Y, 0\right)$

$$E[L_x] = \int_0^1\int_0^1 max\left(y-x,0\right)\frac{x^a\left(1-x\right)^b}{B\left(a,b\right)} \frac{y^c\left(1-y\right)^d}{B\left(c,d\right)}~dx~dy
= \frac{B\left(c+1,d\right)}{B\left(c,d\right)}g\left(c+1,d,a,b\right) - 
\frac{B\left(a+1,b\right)}{B\left(a,b\right)}g\left(c,d,a+1,b\right)$$

$$E[L_y] = \int_0^1\int_0^1 max\left(x-y,0\right)\frac{x^a\left(1-x\right)^b}{B\left(a,b\right)} \frac{y^c\left(1-y\right)^d}{B\left(c,d\right)}~dx~dy
= \frac{B\left(a+1,b\right)}{B\left(a,b\right)}g\left(a+1,b,c,d\right) - 
\frac{B\left(c+1,d\right)}{B\left(c,d\right)}g\left(a,b,c+1,d\right)$$
  
  
Note: the derivation of the above closed-form formula can be found [here](https://www.chrisstucchio.com/blog/2014/bayesian_ab_decision_rule.html).

**Decision rule:**  
  
Chose a threshold of caring($toc$). If the expected loss of a variant is below $toc$, delare this variant as the winner.

Based on the simulation study, [threshold of caring values recommended by VWO](https://vwo.com/knowledge/what-is-smart-decision) normally results in high type 1 error rates(~12% for high certainty values). So more conversative toc values are suggeted for running both fixed sample size tests and sequential tests. A toc calculation example is show below:

In [3]:
# 0.00000250(the value after sequential:) is the suggested toc for running a sequential test.
import util.helper as helper
helper.estimate_toc(p0=0.02, mde=0.05)

TOC for p0=0.020, mde=0.050(lift=0.00100):
VWO recommendation: high certainty: 0.00001000, balance: 0.00007500, quick learning: 0.00020000
Suggested value: fixed sample size:0.00000250, sequential:0.00000050


A simulated example based on the suggested $toc$:

In [4]:
import numpy as np
import util.ab_core as ab_core
np.random.seed(2019)
data = [np.random.binomial(1, 0.02, 521855), np.random.binomial(1, 0.02, 521855)]
ab_core.calc_ab(data, toc=0.00000250) # plug the data into calc_ab to produce output statistics for a test

{'[sa, na, sb, nb]': [10484, 521855, 10378, 521855],
 '[alpha_a, beta_a, alpha_b, beta_b]': [10485, 511372, 10379, 511478],
 'cvr_a': 0.02009171094763508,
 'cvr_b': 0.019888590169337576,
 'ci_cvr_a': [0.019594853523271127, 0.020595477552368346],
 'ci_cvr_b': [0.01939421804504697, 0.020389874397584662],
 'prob_a_gt_b': 0.7707480017080857,
 'prob_b_gt_a': 0.22925199829191434,
 'lift_b_vs_a': -0.0002031207782975039,
 'el_a': 3.648315347919845e-05,
 'el_b': 0.00023960391907216529,
 'el_res': 'U'}

## Runing a Bayesian A/B/C test at a fixed sample size

For running a A/B/C test, we don't have closed-form formula to compute output statistics, so we need to resort to 
Monte Carlo sampling to compute them. Monte Carlo approximation is based on the law of large numbers which states:

$$Random\,sample\,\,X_1,X_2,...X_n\sim X$$
$$E[X]=\frac{1}{n} \sum_{i=1}^{n} X_i\,\,\,\,\,when\,\,n\,is\,large$$
$$E[g\left(X\right)]=\frac{1}{n} \sum_{i=1}^{n} g\left(X_i\right)\,\,\,\,\,when\,\,n\,is\,large$$

Below is an example of using Monte Carlo sampling to compute an approximate mean of $Beta\left(2,000,8,000\right)$.

In [5]:
import numpy as np
import scipy.stats as stats
np.random.seed(2019)
# draw 100,000 samples from Beta(2000, 8000)
rvs_a = stats.beta.rvs(2000, 8000, size=100000)
print("Exact mean: {}\nMonte Carlo approximate mean: {}".format(stats.beta(2000, 8000).mean(),np.mean(rvs_a)))

Exact mean: 0.2
Monte Carlo approximate mean: 0.1999846840380408


Like A/B testing, we first compute the required sample size for 3 variants testing and estimate the threshold of caring($toc$).

In [6]:
import util.helper as helper
helper.sample_size(alpha=0.05, beta=0.05, p0=0.02, mde=0.05, variants=3), print()
helper.estimate_toc(p0=0.02, mde=0.05)

Sample size for alpha=0.050, beta=0.050, p0=0.020, mde=0.050(lift=0.001):
Each variant=521,855, Total=1,565,564

TOC for p0=0.020, mde=0.050(lift=0.00100):
VWO recommendation: high certainty: 0.00001000, balance: 0.00007500, quick learning: 0.00020000
Suggested value: fixed sample size:0.00000250, sequential:0.00000050


**The loss function and the decision rule for 3 variant tests are defined as below:**

**Loss fuction:**

$X:\,Random\,variable\,for\:conversion\,rate\,of\,A\,$  
$\,Y:\,Random\,variable\,for\,conversion\,rate\,of\,B$  
$\,Z:\,Random\,variable\,for\,conversion\,rate\,of\,C$  
  
$L_x = max\left(Z-X, Y-X, 0\right), L_y = max\left(X-Y, Z-Y, 0\right), L_z = max\left(X-Z, Y-Z, 0\right)$  

**Decision rule:** 

Chose a threshold of caring($toc$). If the expected loss of a variant is below $toc$, delare this variant as the winner
    
       
A simulated example based on the suggested $toc$:

In [7]:
import numpy as np
import util.abc_core_mc as abc_core

np.random.seed(2019)
data = [np.random.binomial(1, 0.02, 521855), 
        np.random.binomial(1, 0.021, 521855),
        np.random.binomial(1, 0.022, 521855)]
abc_core.calc_abc(data, toc=0.00000250)

{'[sa, na, sb, nb, sc, nc]': [10484, 521855, 10886, 521855, 11568, 521855],
 '[alpha_a, beta_a, alpha_b, beta_b, alpha_c, beta_c]': [10485,
  511372,
  10887,
  510970,
  11569,
  510288],
 'cvr_a': 0.02009171094763508,
 'cvr_b': 0.020862036918159573,
 'cvr_c': 0.022168908340790678,
 'ci_cvr_a': [0.019594853523271127, 0.020595477552368346],
 'ci_cvr_b': [0.020355873465986066, 0.021375098463006004],
 'ci_cvr_c': [0.02164736663798896, 0.022697329325373383],
 'prob_b_gt_a': 0.99775,
 'prob_c_gt_a': 1.0,
 'prob_a_beat_all': 0.0,
 'prob_b_beat_all': 0.0,
 'prob_c_beat_all': 1.0,
 'lift_b_vs_a': 0.0007703259705244929,
 'lift_c_vs_a': 0.0020771973931555977,
 'el_a': 0.0020770183577265827,
 'el_b': 0.0013062452273601146,
 'el_c': 0.0,
 'el_res': 'C'}

**The logic and code for A/B/C testing can be easily extended for A/B/n testing.** Multiple comparision correction issue is not mentioned in the paper. 
My inistial simulation study shows that VWO's Bayesian approach doesn't suffer from this issue. 

## Running a Bayesian sequential test

In order to control type 1 error rates, we need to use even more conservative $toc$ for sequential testing. 
For given $p_0$ and $mde$, the suggested toc is $p_0*mde*0.0005$. The simulation study shows that this suggested 
value can limit the type 1 error rates under 5% after 20 peeks while the power is resuced by 5% compared to t-test.

The advantage of sequential testing is that we can just run the test without a predetermined sample size and only
need to estimate the $toc$ prior to the start of a test.

In [8]:
# 0.00000050(the value after sequential:) is the suggested toc for running a sequential test.
import util.helper as helper
helper.estimate_toc(p0=0.02, mde=0.05) 

TOC for p0=0.020, mde=0.050(lift=0.00100):
VWO recommendation: high certainty: 0.00001000, balance: 0.00007500, quick learning: 0.00020000
Suggested value: fixed sample size:0.00000250, sequential:0.00000050


In [9]:
import numpy as np
import util.sim_ab as sim_ab
p0, p1 = 0.02, 0.02
data = helper.sim_binomial_seq([p0, p1], seq_len=521855, samples=1000)
el_res, el_len = sim_ab.sim_peeking(data, toc=0.00000050, start=20000, step=25000)
print("VWO/Peeking - Type 1 error rate: {:.4f}".format(1-el_res['U']/1000))

peek #1 at first 20000 data points, EL_res: {'U': 997, 'A': 3}
peek #2 at first 45000 data points, EL_res: {'U': 991, 'A': 8, 'B': 1}
peek #3 at first 70000 data points, EL_res: {'U': 985, 'A': 11, 'B': 4}
peek #4 at first 95000 data points, EL_res: {'U': 982, 'A': 13, 'B': 5}
peek #5 at first 120000 data points, EL_res: {'U': 980, 'A': 15, 'B': 5}
peek #6 at first 145000 data points, EL_res: {'U': 975, 'A': 16, 'B': 9}
peek #7 at first 170000 data points, EL_res: {'U': 970, 'A': 20, 'B': 10}
peek #8 at first 195000 data points, EL_res: {'U': 968, 'A': 20, 'B': 12}
peek #9 at first 220000 data points, EL_res: {'U': 963, 'A': 21, 'B': 16}
peek #10 at first 245000 data points, EL_res: {'U': 962, 'A': 21, 'B': 17}
peek #11 at first 270000 data points, EL_res: {'U': 962, 'A': 21, 'B': 17}
peek #12 at first 295000 data points, EL_res: {'U': 960, 'A': 22, 'B': 18}
peek #13 at first 320000 data points, EL_res: {'U': 958, 'A': 23, 'B': 19}
peek #14 at first 345000 data points, EL_res: {'U': 95

In [10]:
import numpy as np
import util.sim_ab as sim_ab
p0, p1 = 0.02, 0.021
data = helper.sim_binomial_seq([p0, p1], seq_len=521855, samples=1000)
el_res, el_len = sim_ab.sim_peeking(data, toc=0.00000050, start=20000, step=25000)
print("VWO/Peeking - Power: {:.4f}".format(el_res['B']/1000))

peek #1 at first 20000 data points, EL_res: {'U': 985, 'B': 15}
peek #2 at first 45000 data points, EL_res: {'U': 957, 'B': 43}
peek #3 at first 70000 data points, EL_res: {'U': 900, 'B': 100}
peek #4 at first 95000 data points, EL_res: {'U': 859, 'B': 141}
peek #5 at first 120000 data points, EL_res: {'U': 800, 'B': 200}
peek #6 at first 145000 data points, EL_res: {'U': 738, 'B': 262}
peek #7 at first 170000 data points, EL_res: {'U': 683, 'B': 317}
peek #8 at first 195000 data points, EL_res: {'U': 619, 'B': 381}
peek #9 at first 220000 data points, EL_res: {'U': 553, 'B': 447}
peek #10 at first 245000 data points, EL_res: {'B': 505, 'U': 495}
peek #11 at first 270000 data points, EL_res: {'B': 556, 'U': 444}
peek #12 at first 295000 data points, EL_res: {'B': 595, 'U': 405}
peek #13 at first 320000 data points, EL_res: {'B': 647, 'U': 353}
peek #14 at first 345000 data points, EL_res: {'B': 700, 'U': 300}
peek #15 at first 370000 data points, EL_res: {'B': 735, 'U': 265}
peek #16 a